In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import matplotlib.pyplot as plt
import requests
import pprint
import sklearn
import gensim
import datetime
import json
import pickle
import fastparquet

seed = 3

import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/DV_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')

INFO:root:good day to you madam fiona


Code from:
- https://sfengc7.stern.nyu.edu:8888/notebooks/patentdiffusion/201801KnowledgeSpilloversRep/MCSampleRegression/1-Doc2Vec.ipynb
- Documentation here: https://radimrehurek.com/gensim/models/doc2vec.html, see example here: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/doc2vec-lee.ipynb

In [ ]:
pats = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas().sample(frac = 0.7, random_state = seed)
abs_stemmed = pats["abs_stemmed"].tolist()
pat_labels = pats["patent"].astype(str).tolist()
del(pats)

In [ ]:
from gensim import models
from gensim.models.doc2vec import TaggedDocument

class DocIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            # print 'creating tagged document...%d' % idx
            yield TaggedDocument(words=doc.split(), tags=[self.labels_list[idx]])


iterator = DocIterator(abs_stemmed, pat_labels)
print('building vocabulary')

%time model = gensim.models.Doc2Vec(size=100, window=10, min_count=50, workers=4, alpha=0.025, min_alpha=0.025)
%time model.build_vocab(iterator)

print('done building vocabulary')
print('start training the model')

starttime = datetime.datetime.now()
print("start")
print(starttime)
model.train(iterator, total_examples=model.corpus_count, epochs=30)
endtime = datetime.datetime.now()
print("end")
print(endtime)

model.save("DataStore/2018-07-P2/ML/doc2vec_0712.model")
print("Finished saving model")
print(datetime.datetime.now())

In [ ]:
## Not doing all patents anymore
# docvecs = []
# for i in iter(pat_labels):
#     docvecs.append(model.docvecs[i])
    
# dv = pd.DataFrame(docvecs, columns = [str(i) for i in range(100)])
# dv.index = [int(i) for i in pat_labels]
# fastparquet.write("DataStore/2018-07-P2/ML/docvecs_pats_0712.parq", dv, compression="GZIP")

In [ ]:
# model = gensim.models.doc2vec.Doc2Vec.load("DataStore/2018-07-P2/ML/doc2vec_0712.model")
pa = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["abs_stemmed"])["abs_stemmed"].tolist()
try:
    texts = [t.split() for t in pa]
    print("inferring new vectors")
    print(datetime.datetime.now())
    new_vecs = [model.infer_vector(t) for t in texts]
    print("finished inferring new vectors")
    print(datetime.datetime.now())
    nv = pd.DataFrame(new_vecs, columns = [str(i) for i in range(100)], index = range(len(new_vecs)))
    fastparquet.write("DataStore/2018-07-P2/ML/docvecs_pats_0712.parq", nv, compression = "GZIP")
except Exception as e:
    logging.exception("message")